In [1]:
import numpy as np
import h5py 
import hdf5storage
import scipy.io as sio
from PIL import Image 
import scipy.misc as sm

In [5]:
nyud_dir = '/home/yongyang/CVProjects/nyud/'

In [4]:
#f= sio.loadmat('/home/yongyang/Desktop/nyud/data/nyu_depth_v2_labeled.mat')
f = h5py.File('/home/yongyang/Desktop/nyud/data/nyu_depth_v2_labeled.mat')

In [4]:
f.keys()

[u'#refs#',
 u'#subsystem#',
 u'accelData',
 u'depths',
 u'images',
 u'instances',
 u'labels',
 u'names',
 u'namesToIds',
 u'rawDepthFilenames',
 u'rawDepths',
 u'rawRgbFilenames',
 u'sceneTypes',
 u'scenes']

## Deal with Depth information, generate img_xxxx.mat and img_xxxx.png

In [28]:
## scale to 0~255
def scale_to_image(array,min_v,max_v):
    return (array -min_v)/(max_v-min_v)*255.

In [5]:
print f['depths'].shape
f['depths'][0]

(1449, 640, 480)


array([[ 134.,  134.,  134., ...,   56.,   56.,   56.],
       [ 134.,  134.,  134., ...,   56.,   56.,   56.],
       [ 134.,  134.,  134., ...,   56.,   56.,   56.],
       ..., 
       [  62.,   62.,   62., ...,   40.,   40.,   40.],
       [  62.,   62.,   62., ...,   40.,   40.,   40.],
       [  62.,   62.,   62., ...,   40.,   40.,   40.]], dtype=float32)

In [6]:
depths_set = np.array(f['depths'])

In [32]:
# find the max value and min value of depths
depths_set = np.array(f['depths'])
max_v = depths_set.max()
mean_v = depths_set.mean()
min_v = depths_set.min()
print 'max vale %f '%max_v,'min value %f '%min_v, 'mean value %f '%mean_v

max vale 255.000000  min value 0.000000  mean value 106.768013 


In [180]:
for i in xrange(1449):
    array = depths_set[i].T
    array = scale_to_image(array,min_v,max_v)
    array = sm.imresize(array,(425,560))           # imresize sholud put at the last position
    im = Image.fromarray(array)
    im = im.convert('RGB')
    im.save(nyud_dir+'data/depths/img_{}.png'.format(5001+i))
    sio.savemat(nyud_dir+'data/depths/img_{}.mat'.format(5001+i),{'depths':array,'dims':'HW'})

In [34]:
img = sio.loadmat(nyud_dir+"data/depths/img_5001.mat")
print img['depths'].shape
print img['dims']

(425, 560)
[u'HW']


## Deal with hha ,resize and rescale, genenate png.

In [35]:
# Find the max,min, mean in x,y,z channel
max_x,min_x,mean_x = 0.,500.,0.
max_y,min_y,mean_y = 0.,500.,0.
max_z,min_z,mean_z = 0.,500.,0.
x_sum_means = 0.
y_sum_means = 0.
z_sum_means = 0.
for i in xrange(1449): # for save memory,process one by one
    hha = sio.loadmat(nyud_dir+'data/pointCloud/img_{}.mat'.format(5001+i))
    x = hha['x3']
    x = np.nan_to_num(x)
    y = hha['y3']
    y = np.nan_to_num(y)
    z = hha['z3']
    z = np.nan_to_num(z)
    # x channel
    x_max_single = np.nanmax(x)
    x_min_single =np.nanmin(x)
    if x_max_single > max_x:max_x = x_max_single
    if x_min_single < min_x:min_x = x_min_single
    x_sum_means += np.nanmean(x)
    # y channel
    y_max_single = np.nanmax(y)
    y_min_single =np.nanmin(y)
    if y_max_single > max_y:max_y = y_max_single
    if y_min_single < min_y:min_y = y_min_single
    y_sum_means += np.nanmean(y)
    # z channel
    z_max_single = np.nanmax(z)
    z_min_single = np.nanmin(z)
    if z_max_single > max_z:max_z = z_max_single
    if z_min_single < min_z:min_z = z_min_single
    z_sum_means += np.nanmean(z)
mean_x = x_sum_means/1449.
mean_y = y_sum_means/1449.
mean_z = z_sum_means/1449.
print 'x channel: ','max %f '%max_x,'min %f '%min_x, 'mean %f '%mean_x
print 'y channel: ','max %f '%max_y,'min %f '%min_y, 'mean %f '%mean_y
print 'z channel: ','max %f '%max_z,'min %f '%min_z, 'mean %f '%mean_z


x channel:  max 531.293210  min -542.521566  mean -1.643854 
y channel:  max 208.502199  min -401.823958  mean -8.740328 
z channel:  max 1003.450286  min 0.000000  mean 238.383164 


In [36]:
print scale_to_image(mean_x,max_x,min_x)
print scale_to_image(mean_y,max_y,min_y)
print scale_to_image(mean_z,max_z,min_z)

126.557162631
90.7659680613
194.421306903


In [39]:
# Check hha dataset
x_sum = 0.
y_sum = 0.
z_sum = 0.
for i in xrange(1449):
    hha = sio.loadmat(nyud_dir+'data/pointCloud/img_{}.mat'.format(5001+i))
    x = hha['x3']
    y = hha['y3']
    z = hha['z3']
    x = scale_to_image(x,min_x,max_x)
    y = scale_to_image(y,min_y,max_y)
    z = scale_to_image(z,min_z,max_z)
    array =np.array([x,y,z])
    vec = array.transpose(1,2,0)
    vec = np.nan_to_num(vec)
    vec = sm.imresize(vec,(425,560))
    im = Image.fromarray(vec)
    im = im.convert('RGB')
    im.save(nyud_dir+'data/hha/img_{}.png'.format(5001+i))
    sio.savemat(nyud_dir+'data/hha/img_{}.mat'.format(5001+i),{'hha':array,'dims':'CHW'})

## Deal with color images,generate image_xxxx.png and image_xxxx.mat

In [130]:
# Helper function for deprocessing preprocessed images, e.g., for display.
def deprocess_net_image(image):
    image = image.copy()              # don't modify destructively
    image = image[::-1]               # BGR -> RGB
    image = image.transpose(2, 1, 0)  # CHW -> HWC
    image += [123, 117, 104]          # (approximately) undo mean subtraction

    # clamp values in [0, 255]
    image[image < 0], image[image > 255] = 0, 255

    # round and cast from float32 to uint8
    image = np.round(image)
    image = np.require(image, dtype=np.uint8)

    return image

In [41]:
print f['images'].shape
f['images'][1448]

(1449, 3, 640, 480)


array([[[255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        ..., 
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255]],

       [[255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        ..., 
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255]],

       [[255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        ..., 
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255]]], dtype=uint8)

In [58]:
import matplotlib.pyplot as plt
for i in xrange(1449):
    # to png
    vec = f["images"][i]         # RGB,CWH
    array =  vec[::-1]           # RGB ->BGR
    array = array.transpose(0,2,1) #CWH ->HWC
    vec = vec.transpose(2, 1,0)  # CWH -> HWC
    vec = sm.imresize(vec,(425,560))
    im = Image.fromarray(vec)
    im = im.convert('RGB')
    im.save(nyud_dir+'data/images/img_{}.png'.format(5001+i))
    # to mat BGR form
    array = vec.transpose(2,0,1)  
    sio.savemat(nyud_dir+'data/images/img_{}.mat'.format(5001+i),{'image':array,'dims':'BGR,CHW'})

In [51]:
img = sio.loadmat(nyud_dir+"data/images/img_5001.mat")
print img['image'].shape
print img['dims']

(3, 425, 560)
[u'BGR,CHW']


## Deal with segmentation label, generate mat file,HW

In [225]:
print f['labels'].shape
print f['labels'][0]

(1449, 640, 480)
[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]


In [163]:
pic_Label = sio.loadmat(nyud_dir+'data/benchmarkData/groundTruth/img_5001.mat')

In [164]:
pic_Label.keys()

['__version__', '__header__', 'groundTruth', '__globals__']

In [224]:
vec = pic_Label['groundTruth']
print vec[0][0][0][0][0].shape
print vec[0][0][0][0][0][0]

(425, 560)
[31 31 31 31 31 31 31 31 31 31 31 31 31 31 31 31 31 31 31 31 31 31 31 31 29
 29 29 29 29 29 29 29 29 29 29 29 30 30 30 30 30 30 30 30 30 30 30 30 30 30
 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30
 30 30 30 30 30 30 30 30 30 30 30 30 30  6  6  6  6  6  6  6  6  6  6  6  6
  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6
  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6
  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6
  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6
  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6
  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6
  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6
  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6
  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6
 

In [248]:
for i in xrange(1449):
    mat = sio.loadmat(nyud_dir+'data/benchmarkData/groundTruth/img_{}.mat'.format(5001+i))
    vec = mat['groundTruth'][0][0][0][0][0]
    #im = Image.fromarray(vec)
    #im.convert('RGB')
    #im.save(nyud_dir+'data/segmentation/img_{}.png'.format(5001+i))
    sio.savemat(nyud_dir+'data/segmentation/img_{}.mat'.format(5001+i),{'segmentation':vec,'dims':'HW'})

In [249]:
# Test segmentation data
mat = sio.loadmat(nyud_dir+'data/segmentation/img_4.mat')
print mat['segmentation']
print mat['dims']

[[31 31 31 ...,  5  5  5]
 [31 31 31 ...,  5  5  5]
 [31 31 31 ...,  5  5  5]
 ..., 
 [ 7  7  7 ..., 22 22 22]
 [ 7  7  7 ..., 22 22 22]
 [ 7  7  7 ..., 22 22 22]]
[u'HW']


In [115]:
img = sio.loadmat(nyud_dir+"data/segmentation/img_5001.mat")
print img['segmentation'].shape
print img['dims']

(480, 640)
[u'HW']


## Check the scene label and object label

## objects 0 means not labeled

In [140]:
print f['names'].shape

(1, 894)


In [143]:
f['names'][0][6]

<HDF5 object reference>

In [144]:
for i in xrange(894):
    test = f['names']
    st = test[0][i]
    obj = f[st]
    str1 = ''.join(chr(i) for i in obj[:])
    print str1 

book
bottle
cabinet
ceiling
chair
cone
counter
dishwasher
faucet
fire extinguisher
floor
garbage bin
microwave
paper towel dispenser
paper
pot
refridgerator
stove burner
table
unknown
wall
bowl
magnet
sink
air vent
box
door knob
door
scissor
tape dispenser
telephone cord
telephone
track light
cork board
cup
desk
laptop
air duct
basket
camera
pipe
shelves
stacked chairs
styrofoam object
whiteboard
computer
keyboard
ladder
monitor
stand
bar
motion camera
projector screen
speaker
bag
clock
green screen
mantel
window
ball
hole puncher
light
manilla envelope
picture
mail shelf
printer
stapler
fax machine
folder
jar
magazine
ruler
cable modem
fan
file
hand sanitizer
paper rack
vase
air conditioner
blinds
flower
plant
sofa
stereo
books
exit sign
room divider
bookshelf
curtain
projector
modem
wire
water purifier
column
hooks
hanging hooks
pen
electrical outlet
doll
eraser
pencil holder
water carboy
mouse
cable rack
wire rack
flipboard
map
paper cutter
tape
thermostat
heater
circuit breaker box

In [42]:
print f['scenes'].shape
f['scenes'][0]

(1, 1449)


array([<HDF5 object reference>, <HDF5 object reference>,
       <HDF5 object reference>, ..., <HDF5 object reference>,
       <HDF5 object reference>, <HDF5 object reference>], dtype=object)

In [43]:
f['sceneTypes'][0][6]

<HDF5 object reference>

In [235]:
for i in xrange(10):
    test = f['scenes']
    st = test[0][i]
    obj = f[st]
    str1 = ''.join(chr(i) for i in obj[:])
    print str1 

kitchen_0004
kitchen_0004
office_0003
office_0003
office_0004
office_0004
office_0004
office_0004
office_0005
office_0006


In [236]:
for i in xrange(5):
    test = f['sceneTypes']
    st = test[0][2]
    obj = f[st]
    str1 = ''.join(chr(i) for i in obj[:])
    print str1 

office
office
office
office
office


## Check the guta meta data

## In total 894 classes of objects, map them to 40 classes in classmap40

In [66]:
classMap04 = sio.loadmat(nyud_dir+'data/benchmarkData//metadata/classMapping04.mat')
classMap40 = sio.loadmat(nyud_dir+'data/benchmarkData/metadata/classMapping40.mat')
nyusplits = sio.loadmat(nyud_dir+'data/benchmarkData/metadata/nyusplits.mat')
sceneMap = sio.loadmat(nyud_dir+'data/benchmarkData/metadata/sceneMapping10.mat')


In [68]:
classMap04.keys()

['mapToNYUSuperClassMat',
 '__header__',
 '__globals__',
 'className',
 'mapToNYUSuperClass',
 'mapClass',
 '__version__',
 'allClassName']

In [228]:
classMap40.keys()

['mapToNYUSuperClassMat',
 '__header__',
 '__globals__',
 'className',
 'mapToNYUSuperClass',
 'mapClass',
 '__version__',
 'allClassName']

In [234]:
classMap40['mapToNYUSuperClass']

array([[2, 1, 3, 3, 3, 3, 3, 2, 2, 3, 4, 2, 2, 3, 3, 2, 3, 4, 4, 4, 4, 2,
        4, 3, 4, 4, 4, 2, 4, 2, 4, 3, 3, 4, 4, 3, 4, 2, 3, 4]], dtype=uint8)

In [237]:
classMap40['allClassName'].shape

(1, 894)

In [131]:
classMap40['allClassName'].shape


(1, 894)

In [76]:
nyusplits

{'__globals__': [],
 '__header__': 'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Mar 10 22:18:44 2013',
 '__version__': '1.0',
 'test': array([[5001, 5002, 5009, 5014, 5015, 5016, 5017, 5018, 5021, 5028, 5029,
         5030, 5031, 5032, 5033, 5034, 5035, 5036, 5037, 5038, 5039, 5040,
         5041, 5042, 5043, 5046, 5047, 5056, 5057, 5059, 5060, 5061, 5062,
         5063, 5076, 5077, 5078, 5079, 5084, 5085, 5086, 5087, 5088, 5089,
         5090, 5091, 5117, 5118, 5119, 5125, 5126, 5127, 5128, 5129, 5131,
         5132, 5133, 5134, 5137, 5153, 5154, 5155, 5167, 5168, 5169, 5171,
         5172, 5173, 5174, 5175, 5176, 5180, 5181, 5182, 5183, 5184, 5185,
         5186, 5187, 5188, 5189, 5190, 5191, 5192, 5193, 5194, 5195, 5196,
         5197, 5198, 5199, 5200, 5201, 5202, 5207, 5208, 5209, 5210, 5211,
         5212, 5220, 5221, 5222, 5250, 5264, 5271, 5272, 5273, 5279, 5280,
         5281, 5282, 5283, 5284, 5285, 5296, 5297, 5298, 5299, 5300, 5301,
         5302, 5310, 5311, 53

In [77]:
sceneMap

{'__globals__': [],
 '__header__': 'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Mon Apr 15 13:06:49 2013',
 '__version__': '1.0',
 'allSceneName': array([[array([u'basement'], 
       dtype='<U8'),
         array([u'bathroom'], 
       dtype='<U8'),
         array([u'bedroom'], 
       dtype='<U7'),
         array([u'bookstore'], 
       dtype='<U9'),
         array([u'cafe'], 
       dtype='<U4'),
         array([u'classroom'], 
       dtype='<U9'),
         array([u'computer_lab'], 
       dtype='<U12'),
         array([u'conference_room'], 
       dtype='<U15'),
         array([u'dinette'], 
       dtype='<U7'),
         array([u'dining_room'], 
       dtype='<U11'),
         array([u'excercise_room'], 
       dtype='<U14'),
         array([u'foyer'], 
       dtype='<U5'),
         array([u'furniture_store'], 
       dtype='<U15'),
         array([u'home_office'], 
       dtype='<U11'),
         array([u'home_storage'], 
       dtype='<U12'),
         array([u'indoor_balcony

In [80]:
imgAllScene = sio.loadmat(nyud_dir+'data/benchmarkData/sceneClassification/imgAllScene.mat')

In [82]:
imgAllScene

{'__globals__': [],
 '__header__': 'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Thu Apr  4 11:21:19 2013',
 '__version__': '1.0',
 'gtScene': array([[17, 17, 20, ..., 10, 10, 10]], dtype=uint8),
 'sceneName': array([[array([u'basement'], 
       dtype='<U8'),
         array([u'bathroom'], 
       dtype='<U8'),
         array([u'bedroom'], 
       dtype='<U7'),
         array([u'bookstore'], 
       dtype='<U9'),
         array([u'cafe'], 
       dtype='<U4'),
         array([u'classroom'], 
       dtype='<U9'),
         array([u'computer_lab'], 
       dtype='<U12'),
         array([u'conference_room'], 
       dtype='<U15'),
         array([u'dinette'], 
       dtype='<U7'),
         array([u'dining_room'], 
       dtype='<U11'),
         array([u'excercise_room'], 
       dtype='<U14'),
         array([u'foyer'], 
       dtype='<U5'),
         array([u'furniture_store'], 
       dtype='<U15'),
         array([u'home_office'], 
       dtype='<U11'),
         array([u'home_stor

In [106]:
imgAllScene['gtScene'][0][0]

17

# the label is from 1 to 27
# but the string array index is from 0 to  26

In [3]:
allSceneLabel =sio.loadmat('/home/yongyang/CVProjects/nyud/data/sceneLabel/imgAllScene.mat')

In [6]:
imgAllScene = sio.loadmat(nyud_dir+'data/benchmarkData/sceneClassification/imgAllScene.mat')

In [6]:
allSceneLabel['gtScene'][0][int('5001')-5001]

17

In [15]:
import pandas as pd
import numpy as np

In [16]:
train_txt = pd.read_csv('/home/yongyang/CVProjects/nyud/data/train.txt',header = None)
val_txt = pd.read_csv('/home/yongyang/CVProjects/nyud/data/val.txt',header = None)
test_txt = pd.read_csv('/home/yongyang/CVProjects/nyud/data/test.txt',header = None)

for  i in range(len(tran_txt)):
    train_lists[allSceneLabel['gtScene'][train_txt[i]-5001]].append(train_txt[i])
for  i in range(len(val_txt)):
    val_lists[allSceneLabel['gtScene'][val_txt[i]-5001]].append(val_txt[i])
for  i in range(len(test_txt)):
     train_lists[allSceneLabel['gtScene'][train_txt[i]-5001]].append(train_txt[i])

In [17]:
train_arr = np.zeros(27)
val_arr = np.zeros(27)
test_arr = np.zeros(27)
for  i in range(len(train_txt)):
    train_arr[allSceneLabel['gtScene'][0][train_txt[0][i]-5001]-1] +=1
for  i in range(len(val_txt)):
    val_arr[allSceneLabel['gtScene'][0][val_txt[0][i]-5001]-1] +=1
for  i in range(len(test_txt)):
    test_arr[allSceneLabel['gtScene'][0][test_txt[0][i]-5001]-1] +=1

In [18]:
allSceneLabel['gtScene'][0][448]

23

In [29]:
print len(train_txt)
print train_arr

715
[   3.   60.  191.   18.    2.   24.    3.    2.    2.   58.    1.    2.
   13.   25.    2.    1.  112.    1.  110.   39.    5.   15.    1.    8.
    2.   12.    3.]


In [26]:
print len(val_txt)
print val_arr

359
[  2.  30.  96.   9.   1.  12.   1.   1.   1.  29.   1.   1.   7.  12.   1.
   0.  56.   1.  55.  19.   2.   8.   1.   4.   1.   6.   2.]


In [28]:
print len(test_txt)
print test_arr

375
[  2.  31.  96.   9.   2.  13.   2.   2.   1.  30.   1.   1.   7.  13.   2.
   1.  57.   1.  56.  20.   3.   8.   1.   5.   2.   7.   2.]


In [21]:
all_data = train_arr+val_arr+test_arr

In [22]:
print all_data

[   7.  121.  383.   36.    5.   49.    6.    5.    4.  117.    3.    4.
   27.   50.    5.    2.  225.    3.  221.   78.   10.   31.    3.   17.
    5.   25.    7.]


In [23]:
all_data.sum()

1449.0

# Generate training,validation and test splits

In [7]:
import pandas as pd

In [8]:
all_scene_label = allSceneLabel['gtScene'][0]

In [9]:
distribute_list = []
for i in xrange(27):
    distribute_list.append([])
for i in range(all_scene_label.shape[0]):
    distribute_list[all_scene_label[i]-1].append(i)    

train_list = []
val_list = []
test_list = []
for i in xrange(27):
    num  = len(distribute_list[i])
    if num > 0:
        num_train = (num+1)/2   # 0~num_train assigne to train list
        train_list.extend(distribute_list[i][0:num_train])
        
        num_val = (num-num_train+1)/2     # num_train~(num_train+num_val) assign to val list
        if num_val >0:
            val_list.extend(distribute_list[i][num_train:num_train+num_val])
            test_list.extend(distribute_list[i][num_train+num_val:])  

In [10]:
#randomly split the train,val,test data for each class
import random
train_list = []
val_list = []
test_list = []
for i in xrange(27):
    num  = len(distribute_list[i])
    if num > 0:
        random.shuffle(distribute_list[i])
        num_train = num/2   # 0~num_train assigne to train list
        train_list.extend(distribute_list[i][0:num_train])
        num_val = (num-num_train)/2     # num_train~(num_train+num_val) assign to val list
        val_list.extend(distribute_list[i][num_train:num_train+num_val])
        test_list.extend(distribute_list[i][num_train+num_val:])  

In [11]:
print len(train_list)
print len(val_list)
print len(test_list)

715
359
375


In [31]:
train_set = np.array(train_list)
train_set += 5001
val_set = np.array(val_list)
val_set += 5001
test_set = np.array(test_list)
test_set += 5001

In [32]:
train_pd = pd.DataFrame(train_set)
train_pd.to_csv('train.txt',header = False,index = None)
train_pd = pd.DataFrame(val_set)
train_pd.to_csv('val.txt',header = False,index = None)
train_pd = pd.DataFrame(test_set)
train_pd.to_csv('test.txt',header = False,index = None)